In [ ]:
import numpy as np
import numpy.linalg as alg

### SVD

In [ ]:
def svd(U_delta):
    """This is not a simple svd, it's the svd we need for our project."""
    N,M = U_delta.shape
    wU = U_delta/np.sqrt(M)
    U,s,V = alg.svd(wU,False)
    S = s**2
    return U,S,V

In [ ]:
A = np.random.rand(20,53)
U,S,V = svd(A)
print(U.shape,S.shape,V.shape)
print(S)

## offline procedure

In [ ]:
class dimension_not_allowed_error(Exception):
    def __init__(self,l,k):
        self.l = l
        self.k = k
    def __str__(self):
        return "An error has occured while adding a new vector of size "+str(self.k)+" in basis of space of size "+str(self.l)

class base_orthonormee:
    
    def __init__(self,l,ps):
        self.Cardinal = 0
        self.Length = l
        self.Scalar_Product = ps
        self.Basis_Set = np.zeros((self.Length,self.Cardinal))
    
    def add(self,v):
        if self.Length != len(v):
            raise dimension_not_allowed_error(self.Length,len(v))
        v -= self.Scalar_Product(self.Basis_Set.T,self.Scalar_Product(self.Basis_Set,v))
        self.Basis_Set = np.concatenate((self.Basis_Set,v/np.sqrt(self.Scalar_Product(v,v))),axis=1)
        self.Cardinal += 1
    
    def matrix(self):
        return self.Basis_Set
    
    def show(self):
        print("vector length : ",self.Length)
        print("cardinal : ",self.Cardinal)
        print("matrix : ",self.Basis_Set)

def offline_procedure(total_model_solver,reduced_basis_solver,ps,pre_computing,training_set,tol,l):
    """ 
    this function takes the problem and a sample of the parameter space and computes the reduced basis
    (following greedy algorithm steps) that approximates the space (V_delta) so that the maximum error is less than the tolerance.
    
        function "total model solver" is the true solver.
                                                Input: mu
                                                Output: u_delta(mu)
        function "reduced basis solver" solves the linear system A^mu_rb . u^mu_rb = f^mu_rb
                                                Input: mu, T (pre_computed elements of A and f)
                                                Output: u_rb(mu), s_rb(mu) = nu(mu)
        function "ps" is the scalar product for normalization and orthogonalization.
                                                Input: 2 vectors
                                                Output: scalar_product result
        function "pre computing" performs the precomputing step
                                                Input: V,n (current V_rb basis)
                                                Ouput: T (n partial A-matrices and f-vectors from affine assumption)
        array "training set" is the sample of the parameter space
        
        float "tol" is the tolerance for stopping criteria
        
        integer "l" determines current vector length
        
    """
    mu = training_set[0]
    max_nu = np.Inf
    V = base_orthonormee(l,ps)
    while max_nu > tol:
        #gram-schmidt pour que V ressemble à une base
        V.add(total_model_solver(mu))
        #pre-computing
        T = pre_computing(V.matrix)
        #calcul de u_rb(mu) pour tout les mu
        max_nu = 0
        for mu_loop in training_set :
            u_rb,nu = reduced_basis_solver(T,mu_loop)
            if max_nu < nu and nu > tol:
                max_nu = nu
                mu = mu_loop
    return V
        

In [ ]:
def ps(u,v):
    return u.T.dot(v)

In [ ]:
## Test base orthonormee
#v1 = np.zeros((3,1))
#v2 = np.zeros((3,1))
#v3 = np.zeros((3,1))
#v1 = np.asarray([[1.],[1.],[1.]])
#v2 = np.asarray([[1.],[-1.],[0.]])
#v3 = np.asarray([[-1.],[-9.],[-9.]])
#test = base_orthonormee(v1,ps)
#test.show()
#test.add(v2)
#test.show()
#test.add(v3)
#test.show()